<h1><center> Network/Graph Analysis in Python </center></h1>

**NetworkX**: Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

Installation: 
`$ pip install networkx`


**nxviz**: network visualization package

Installation:
`$ pip install nxviz`


If already installed, to upgrade to latest versions: `$ pip install networkx --upgrade` and `$ pip install nxviz --upgrade`

**Some open source network data locations:**

* **graph examples embedded in NetworkX:**
    * Zachary's Karate Club: `nx.karate_club_graph()`
    * Davis Southern women social network: `nx.davis_southern_women_graph()`
    *  Florentine families: `nx.florentine_families_graph()`
    * more examples: https://networkx.github.io/documentation/networkx-1.9/examples/index.html
* **The Koblenz Network Collection**: http://konect.uni-koblenz.de/
* **Stanford Large Network Dataset Collection**: https://snap.stanford.edu/data/


**To learn the basics of Network Science: http://networksciencebook.com/**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import networkx as nx
import nxviz as nv
%matplotlib inline

In [ ]:
# check version 
print('NetworkX: ', nx.__version__)
print('nxviz: ', nv.__version__)

## I. Networks Basics

**Nodes**: can represent anything (images, webpage URL links, people, power stations, numbers, words, etc.)

**Edges**: represent relationships between nodes

### 1. Create & Manipulate Networks 

In [ ]:
# create empty network
G = nx.Graph()

In [ ]:
# add one node with label '1'
G.add_node(1)

In [ ]:
# add nodes from a list of elements
G.add_nodes_from(['Mary', 4, 'Alice', 'Mary'])

In [ ]:
# remove node
G.remove_node('Mary')

In [ ]:
# remove multiple nodes
G.remove_nodes_from(['Mary', 1])

In [ ]:
# view nodes in network G
G.nodes

In [ ]:
# add single edge - tuple of nodes (source, target)
# this also adds nodes if they don't already exist
G.add_edge('Mary','Steven')

In [ ]:
G.nodes

In [ ]:
# add multiple of edges (list of tuples)
G.add_edges_from([('Mary', 'Steven') , ('Mary', 'Alice')])

In [ ]:
# view edges in network G
G.edges

In [ ]:
# remove edge
G.remove_edge('Mary','Alice')

In [ ]:
# remove multiple edges (list of tuples)
G.remove_edges_from([('Mary', 'Steven') , ('Mary', 'Alice')])

In [ ]:
# empty the network
G.clear()

**Load network from pandas dataframe.**

In [ ]:
df = pd.read_csv('../datafiles/social/facebook/facebook_clean_data/politician_edges.csv')

In [ ]:
df.head()

In [ ]:
# load graph from pandas dataframe
G = nx.from_pandas_edgelist(df, source='#node_1', target='node_2', create_using=nx.Graph())

In [ ]:
len(G.nodes)

In [ ]:
len(G.edges)

In [ ]:
# graph edgelist to pandas dataframe
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_g.head()

**Load network from file.** 

You can read/write a graph in a file using common graph formats (edge lists, adjacency lists, GML, GraphML, pickle, LEDA, etc.).

To see how to read different types of adjancency formats, check here: https://networkx.github.io/documentation/networkx-1.10/reference/readwrite.html

### 2. Social Networks - Physicians 

In [ ]:
# load physicians network
G = nx.read_edgelist("../datafiles/social/physicians/out.moreno_innovation_innovation", comments='%')
#G = nx.from_pandas_edgelist(df)

In [ ]:
# get number of nodes in network G
G.number_of_nodes()

In [ ]:
# get number of nodes in network G
len(G.nodes)

In [ ]:
# get number of edges in network G
G.number_of_edges()

In [ ]:
# get number of edges in network G
len(G.edges)

In [ ]:
# get number of neighbors (connections) of a specified node
G.degree('151')

In [ ]:
G.nodes

In [ ]:
# get the 2nd node's neighbors (retrieves a dictionary)
dict_neighbors = G.neighbors('2')

In [ ]:
# output edgelist to file
nx.write_edgelist(G,'physician.edgelist')
#nx.to_pandas_edgelist(G)

### 2. Network Types 

#### a. Weighted Graphs

**Edge weight:** quantifies the strength of the connection

In [ ]:
# assign weight to edge
G.add_edge('Mary','Steven', weight=0.6)

In [ ]:
G.edges

In [ ]:
# access edge properties
G['Mary']['Steven']

In [ ]:
# change edge weight
G['Mary']['Steven']['weight'] = 1

#### b. Directed Graphs

**Edge direction:** describes source -> target node relationship

In [ ]:
#undirected
G.nodes

In [ ]:
dg = nx.DiGraph()

In [ ]:
# you can create an undirected representation of network G
nx.to_undirected(dg)

In [ ]:
# you can create a directed representation of network G
dg = nx.to_directed(G)

In [ ]:
dg.edges

#### c. Multigraphs

Many algorithms are not well defined on such graphs. Therefore, you should convert such graphs rather to a standard graph in a way that makes the measurement well defined.

In [ ]:
# multigraphs can store multiple edges information between same two nodes that can have different properties
MG = nx.MultiGraph()
#MG = nx.MultiDiGraph()
MG.add_weighted_edges_from([(1, 2, 3.0), (1, 2, 75), (2, 3, 5), (1, 2, 4)])

In [ ]:
# lists the edges (node1, node2, edge_index), including the multiedges, adding the multiedge index as 3rd element in edge tuple
MG.edges

In [ ]:
# lists the edges (node1, node2, weight/edge_attribute), the 3rd element is the weights of the edges
MG.edges.data('weight')

In [ ]:
MG.edges.data()

In [ ]:
# check the weight of an edge
MG[1][2]

#### d. Bipartite Network

Bipartite graphs B = (U, V, E) have two node sets U, V and edges in E that only connect nodes from opposite sets.

Taken from NetworkX documentation:
* NetworkX does not have a custom bipartite graph class 
* Graph() or DiGraph() classes can be used to represent bipartite graphs 
* you have to keep track of which set each node belongs to, and make sure that there is no edge between nodes of the same set 
* convetion: use a node attribute named *bipartite* with values 0 or 1 to identify the sets each node belongs to. This convention is not enforced in the source code of bipartite functions, it’s only a recommendation.

For more details: https://networkx.github.io/documentation/stable/reference/algorithms/bipartite.html

<img src="../images/Simple-bipartite-graph.svg.png" alt="Data" style="width: 300px;"/>

In [ ]:
from networkx.algorithms import bipartite

In [ ]:
B = nx.Graph()

# add nodes with the node attribute "bipartite"
B.add_nodes_from([1, 2, 3, 4], bipartite=0)
B.add_nodes_from(["a", "b", "c"], bipartite=1)

# add edges only between nodes of opposite node sets
B.add_edges_from([(1, "a"), (1, "b"), (2, "b"), (2, "c"), (3, "c"), (4, "a")])

In [ ]:
# check if graph is bipartite
nx.is_bipartite(B)

## II. Analysis of Structural Properties

#### 1. Node degree, network average degree, degree distribution

In [ ]:
degrees = [deg for node, deg in nx.degree(G)]

In [ ]:
# kmin - minimum degree
kmin = np.min(degrees)
print("Minimum degree: ", kmin)

# kmax - maximum degree
kmax = np.max(degrees)
print("Maximum degree: ", kmax)

# kavg - average degree of the network
kavg = np.mean(degrees)
print("Average degree: ", kavg)

**Degree distribution**: helps us understand connectivity trends in networks and how edges are distributed among nodes (does everyone have similar number of connections, or do we have hubs, nodes with significantly higher number of connections?)

In [ ]:
def degree_distr(net):
    degrees = dict(net.degree()) 
    hist = list(Counter(degrees.values()).items()) 
    hist.sort(key=lambda x:x[0])
    hist = np.array(hist)
    return hist

In [ ]:
dd = degree_distr(G)

In [ ]:
plt.figure()
plt.loglog(dd.T[0],dd.T[1],'ro-')
plt.legend(['Physicians'])
plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.title('Physicians')

#### 2. Paths on networks: average path length, shortest path, longest path

In [ ]:
# find shortest path between node1 and node2 in directed & undirected networks 
nx.shortest_path(G, '1', '15')

In [ ]:
# average path length in graph
nx.average_shortest_path_length(G, weight=None)

#### 3. Clustering coefficient, triangles

In [ ]:
# triangles
nx.triangles(G)

In [ ]:
# clustering coefficient of a node
nx.clustering(G, '1')

In [ ]:
# clustering coefficient of all nodes (returns a dictionary)
nx.clustering(G)

In [ ]:
# clustering coefficient of the network
cc = nx.clustering(G)
avg_clust = sum(cc.values()) / len(cc)
print("Physicians network clustering coefficient:", avg_clust)

#### 4. Centrality measures

In [ ]:
# degree centrality
nx.degree_centrality(G)

In [ ]:
# betweenness centrality of network
nx.betweenness_centrality(G)

In [ ]:
# closeness centrality of network
nx.closeness_centrality(G)

#### 5. Components

In [ ]:
# checks whether the network is connected
nx.is_connected(G)

In [ ]:
# find number of connected components
nx.number_connected_components(G)

In [ ]:
# get the nodes in the same component as *n*
nx.node_connected_component(G, '1')

#### 6. Assortativity

* Pearson correlation coefficient [-1; 1]
* social networks are highly assortative (homophily): high degree nodes connect to other high degree nodes
* technological are disassortative: high degree nodes connect to low degree nodes

Assortativity computed based on:
* degree
* attribute

In [ ]:
# check assortativity of network
nx.assortativity.degree_pearson_correlation_coefficient(G)

In [ ]:
nx.degree_assortativity_coefficient(G)

In [ ]:
# check assortativity (mixing) by a particular attribute
nx.attribute_assortativity_coefficient(G, attribute)

## III. Network Visualization

Disclaimer: visualizations can be pretty and insightful, however for large networks they tend to be just pretty! Knowing how to compute network measures is the key!

* visualization with NetworkX: *"NetworkX provides basic functionality for visualizing graphs, but its main goal is to enable graph analysis rather than perform graph visualization. In the future, graph visualization functionality may be removed from NetworkX or only available as an add-on package."*

* nxviz
* Matplotlib or Graphviz with pydot (import and export NetworkX graphs in Graphviz dot format using pydot)
* Gephi
* Graphviz
* Neo4j
* D3
* etc.

#### 1. Basic drawing methods in NetworkX module: 
* `nx.draw()`
* `nx.draw_random()`
* `nx.draw_spectral()`
* `nx.draw_circular()`
* `nx.draw_spring()`
* `nx.draw_shell()`

In [ ]:
# nxviz package provides some nice visualization options
import nxviz as nv

In [ ]:
nx.draw(G, node_size=10)

In [ ]:
nx.draw_random(G)

In [ ]:
nx.draw_spectral(G)

In [ ]:
nx.draw_circular(G, edge_color='blue', node_size=10)

#### Visualize subgraphs

In [ ]:
# assign selected subgraph to a new graph
nodes = list(G.neighbors('10'))
nodes.append('10')
G_sub = G.subgraph(nodes)

In [ ]:
# drawing options: set node size, color, labels, etc. (check documentation for more)
nx.draw(G_sub, node_size=50, node_color='y', with_labels=True)

#### Visualize with Nxviz

In [ ]:
nv.CircosPlot(G).draw()

In [ ]:
nv.ArcPlot(G).draw()

In [ ]:
nv.MatrixPlot(G).draw()

#### Network with node attributes

* how to import node attributes located in a separate file

In [ ]:
# lexical network: David Copperfield
G_lex = nx.read_edgelist("../datafiles/lexical/david_copperfield/out.adjnoun_adjacency_adjacency", comments='%')

Nodes are listed as numbers, but when visualizing our network, we would like to see the words that represent those nodes. The words are stored in a separate file, in `ent.adjnoun_adjacency_adjacency.word.name`, and the index of the word corresponds to the index of the node. 

In [ ]:
# load node name data for each node in the network
with open("../datafiles/lexical/david_copperfield/ent.adjnoun_adjacency_adjacency.word.name") as file:
    node_name = {}
    i = 1 
    for line in file:
        node_name[str(i)] = line.strip()
        i += 1

In [ ]:
# assign the name of the node to the node in the graph
nx.set_node_attributes(G_lex, node_name, 'name')

In [ ]:
nx.get_node_attributes(G_lex, 'name')

In [ ]:
nx.draw(G_lex, labels=node_name, node_color='y')

In [ ]:
degrees = [deg for node, deg in nx.degree(G_lex)]

In [ ]:
plt.figure(figsize=(10,10))
nx.draw_circular(G_lex, labels=node_name, node_color=degrees, 
                 node_size=200, edge_color='gray', cmap='Greens')

#### Community detection and visualization

In [ ]:
import community as community_louvain
G = nx.karate_club_graph()
partitions = community_louvain.best_partition(G)
nx.draw(G, node_color=list(partitions.values()))

## IV: Network Features for Machine Learning Models

Network features can be incorporated in machine learning algorithms to improve prediction capabilities.  

- structural properties that provide unique features on the node:
    * degree
    * centrality
    * clustering coefficient, etc. 
    
Network topological measures provide additional information that are connectivity dependent, which are additional features for prediction.